# Proyecto: Análisis del Sentimiento en Criptomonedas y su Relación con el Tipo de Cambio
# Objetivo
Evaluar si existe una correlación significativa entre el sentimiento en el mercado de criptomonedas (medido por el Fear & Greed Index) y el tipo de cambio de monedas emergentes como el sol peruano (PEN/USD).

# **Trabajo 2: Modelado Lineal Supervisado, Validación y Comparación de Modelos**

**Curso:** Introducción a Machine Learning con Python

**Grupo:** N° 8

**Integrantes:**

Luis Ángel Alejandro Arrieta Feria

Mirelli Thais Jimenez Pulache

Néstor Julio Rivero Escobar

**Tema:** ¿Existe una correlación significativa entre el sentimiento en el mercado de criptomonedas (medido por el Fear & Greed Index) y el tipo de cambio de monedas emergentes como el sol peruano?

## 1. Carga de Librerías y Datos

En esta sección se importan las librerías necesarias y se obtienen los datos desde la API de Alternative.me (Fear & Greed Index). También se configuran los parámetros generales y el formato visual para los gráficos

In [ ]:
# ============================
# 1. Importamos librerías
# ============================
import re
import requests # Para hacer solicitudes HTTP y conectarnos al API del BCRP
import pandas as pd # Para manejar y analizar datos en tablas
import numpy as np # Para operaciones matemáticas y numéricas
import yfinance as yf # Para descargar datos financieros de Yahoo Finance
import matplotlib.pyplot as plt # Para crear gráficos básicos
import seaborn as sns # Para visualización estadística avanzada y gráficos mejorados
import warnings
warnings.filterwarnings('ignore')
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10,5)
pd.set_option('display.float_format', lambda x: f'{x:,.6f}')


# Configuración visual
sns.set_style('whitegrid') # Estilo de fondo con cuadrícula clara
plt.style.use('seaborn-v0_8-deep') # Estilo visual con colores definidos por seaborn

# ============================
# 2. Parámetros generales
# ============================
periodo_inicial = "2018-06-01"   # Fecha de inicio de los datos
periodo_final   = "2025-06-30"   # Fecha de fin de los datos

# ============================
# 3. Fear & Greed Index (Alternative.me API)
# ============================
url_fng = "https://api.alternative.me/fng/?limit=0&format=json" # URL de la API del Fear & Greed Index
response_fng = requests.get(url_fng) # Hacemos la solicitud a la API
data_fng = response_fng.json() # Convertimos la respuesta en formato JSON

# Procesamos la data
fng = pd.DataFrame(data_fng['data']) # Transformamos la parte útil en un DataFrame

# Conversión y limpieza
fng['date'] = pd.to_datetime(fng['timestamp'].astype(int), unit='s') # Convertimos timestamp a fechas legibles
fng = fng[['date', 'value', 'value_classification']].rename(  # Seleccionamos columnas clave y renombramos
    columns={'value': 'FGI', 'value_classification': 'classification'}
)
fng['FGI'] = pd.to_numeric(fng['FGI'], errors='coerce') # Convertimos la columna FGI a valores numéricos
fng = fng.sort_values('date').set_index('date') # Ordenamos por fecha y la usamos como índice
fng = fng.loc[periodo_inicial:]  # Cortamos desde inicio (se limitará al final más adelante)

# Resumen
print("FGI:", fng.index.min(), "->", fng.index.max(), "| registros:", len(fng))

# Mostrar tabla en Colab (primeros registros)
from IPython.display import display
display(fng.head(10)) # Mostramos las primeras 10 filas

FGI: 2018-06-01 00:00:00 -> 2025-10-17 00:00:00 | registros: 2695


,FGI,classification
date,,
2018-06-01,24,Extreme Fear
2018-06-02,27,Fear
2018-06-03,40,Fear
2018-06-04,41,Fear
2018-06-05,26,Fear
2018-06-06,42,Fear
2018-06-07,38,Fear
2018-06-08,40,Fear
2018-06-09,39,Fear


## 2. Carga y Preparación de Datos: Tipo de Cambio (BCRP API)

En esta sección se consulta la API del Banco Central de Reserva del Perú (BCRP) para obtener la serie diaria del tipo de cambio venta (USD/PEN).  
Se procesa la información, se convierten las fechas a formato estándar y se construye el DataFrame `tc_df` que servirá como base para el análisis posterior.

In [ ]:
# ============================
# 4. Tipo de Cambio (BCRP API)
# ============================

# Parámetros de la consulta
codigo_bcrp = "PD04640PD"    # Código de la serie: TC Sistema bancario SBS diario - Venta
formato = "json"              # Formato en el que se solicitarán los datos (JSON)

# Construcción del URL y petición GET a la API del BCRP
url_bcrp = f"https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo_bcrp}/{formato}/{periodo_inicial}/{periodo_final}"
response_bcrp = requests.get(url_bcrp)  # Se envía la solicitud a la API
data_bcrp = response_bcrp.json()        # Se convierte la respuesta a formato JSON

# Extraemos los nombres (fechas) y los valores de la respuesta JSON
periodos = [p['name'] for p in data_bcrp['periods']]  # Lista de etiquetas de tiempo (fechas)
valores  = [p['values'][0] if (p['values'] and p['values'][0] not in ['n.d.','']) else None
            for p in data_bcrp['periods']]            # Lista de valores (tipo de cambio diario)

# Mapa de abreviaciones de meses del inglés/español al número correspondiente
mes_map = {
    'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07',
    'Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12',
    'Ene':'01','Abr':'04','Ago':'08','Set':'09','Dic':'12'
}

def parse_bcrp_label(lbl):
    """Convierte etiquetas de fecha del BCRP en un formato de fecha válido (datetime)."""
    lbl = str(lbl).strip()
    dt = pd.to_datetime(lbl, errors='coerce', dayfirst=False)  # Intenta convertir directamente
    if not pd.isna(dt):  # Si la conversión funciona, retorna la fecha
        return dt

    # Si no funcionó, intenta extraer día, mes y año con expresiones regulares
    m = re.match(r"^(\d{1,2})[.\-/ ]+([A-Za-z]{3,})[.\-/ ]+(\d{2,4})$", lbl)
    if m:
        dia, mes_ab, ano = m.groups()
        mes_num = mes_map.get(mes_ab[:3].title(), None)
        if mes_num:
            if len(ano) == 2:
                ano = "20" + ano  # Ajuste para años con dos dígitos
            return pd.to_datetime(f"{ano}-{mes_num}-{int(dia):02d}")

    # Si el formato es solo mes y año (sin día)
    m2 = re.match(r"^([A-Za-z]{3,})[.\-/ ]?(\d{2,4})$", lbl)
    if m2:
        mes_ab, ano = m2.groups()
        mes_num = mes_map.get(mes_ab[:3].title(), None)
        if mes_num:
            if len(ano) == 2:
                ano = "20" + ano
            return pd.to_datetime(f"{ano}-{mes_num}-01")

    # Si el formato es solo el año
    m3 = re.match(r"^(\d{4})$", lbl)
    if m3:
        return pd.to_datetime(f"{m3.group(1)}-01-01")

    # Si no se puede convertir, retorna NaT (fecha vacía)
    return pd.NaT

# Aplicamos la función de parsing a todas las fechas obtenidas del BCRP
fechas_parsed = [parse_bcrp_label(lbl) for lbl in periodos]

# Creamos un DataFrame con las fechas y los valores del tipo de cambio
tc_df = pd.DataFrame({
    'Fecha': fechas_parsed,
    'USD_PEN_Venta': [float(x) if x is not None else np.nan for x in valores]
}).dropna(subset=['Fecha']).set_index('Fecha').sort_index()  # Se eliminan nulos y se ordena por fecha

# Imprimimos un resumen del rango temporal y cantidad de registros
print("TC (BCRP):", tc_df.index.min(), "->", tc_df.index.max(), "| registros:", len(tc_df))

# Mostrar en pantalla los primeros 10 registros del DataFrame (formato tabla de Colab)
from IPython.display import display
display(tc_df.head(10))

TC (BCRP): 2018-06-01 00:00:00 -> 2025-06-30 00:00:00 | registros: 1769


,USD_PEN_Venta
Fecha,
2018-06-01,3.273000
2018-06-04,3.273000
2018-06-05,3.273000
2018-06-06,3.261000
2018-06-07,3.262000
2018-06-08,3.263000
2018-06-11,3.267000
2018-06-12,3.270000
2018-06-13,3.266000


## 3. Carga de Variables de Control (Yahoo Finance)

En esta sección se descargan indicadores financieros internacionales desde *Yahoo Finance*; entre ellos el índice del dólar (DXY), el índice de volatilidad (VIX), el precio del Bitcoin, el Oro, y las tasas de los bonos del Tesoro (13 semanas y 5 años).  
Estas variables servirán como **controles** en los modelos econométricos posteriores, ya que capturan condiciones globales que pueden influir sobre el tipo de cambio.


In [ ]:
# ============================
# 5. Variables de control (Yahoo Finance)
# ============================

# Nota: yfinance excluye 'end', pedimos +1 día y luego recortamos
yf_end = (pd.to_datetime(periodo_final) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')

# Tickers: DXY, VIX, Bitcoin, Oro, T-Bills (13w) y Treasury 5y
tickers = ['DX-Y.NYB', '^VIX', 'BTC-USD', 'GC=F', '^IRX', '^FVX']

control_df = yf.download(tickers, start=periodo_inicial, end=yf_end)['Close']

# Renombramos columnas
control_df.rename(columns={
    'DX-Y.NYB': 'DXY',
    '^VIX': 'VIX',
    'BTC-USD': 'BTC_USD',
    'GC=F': 'Gold',
    '^IRX': 'TBills_13w',
    '^FVX': 'Treasury_5y'
}, inplace=True)

# Recortamos al periodo final y ordenamos
control_df = control_df[control_df.index <= pd.to_datetime(periodo_final)].sort_index()

# Resumen
print("Control vars:", control_df.index.min().date(), "->", control_df.index.max().date(), "| registros:", len(control_df))
print("Última fila (fecha límite):")
display(control_df.tail(10))


[*********************100%***********************]  6 of 6 completed

Control vars: 2018-06-01 -> 2025-06-30 | registros: 2587
Última fila (fecha límite):


Ticker,BTC_USD,DXY,Gold,Treasury_5y,TBills_13w,VIX
Date,,,,,,
2025-06-21,"102,257.406250",NaN,NaN,NaN,NaN,NaN
2025-06-22,"100,987.140625",NaN,NaN,NaN,NaN,NaN
2025-06-23,"105,577.773438",98.419998,"3,377.699951",3.884000,4.158000,19.830000
2025-06-24,"106,045.632812",97.860001,"3,317.399902",3.856000,4.182000,17.480000
2025-06-25,"107,361.257812",97.680000,"3,327.100098",3.847000,4.188000,16.760000
2025-06-26,"106,960.000000",97.150002,"3,333.500000",3.799000,4.193000,16.590000
2025-06-27,"107,088.429688",97.400002,"3,273.699951",3.828000,4.197000,16.320000
2025-06-28,"107,327.703125",NaN,NaN,NaN,NaN,NaN
2025-06-29,"108,385.570312",NaN,NaN,NaN,NaN,NaN



# 6. Construcción del dataset final y exportación a Excel
En esta sección se construye el dataset final unificado que servirá como base para el modelado econométrico posterior.
Primero, se define un calendario diario completo que abarca desde la fecha más temprana disponible en las fuentes hasta el periodo final del análisis. Luego, cada una de las tres bases, el índice de sentimiento financiero (fng), el tipo de cambio (tc_df) y los indicadores de control (control_df), se reindexa siguiendo ese mismo calendario para garantizar que todas compartan una frecuencia temporal común y estén perfectamente alineadas.

Posteriormente, las series se concatenan en un único DataFrame, que incluye las variables clave:

FGI (Fear and Greed Index), USD_PEN_Venta (tipo de cambio venta), y los controles financieros (DXY, VIX, BTC_USD, Gold, TBills_13w, Treasury_5y).

Finalmente, se realiza una verificación del rango temporal cubierto y del número de observaciones disponibles por variable. Esto permite confirmar que el dataset esté limpio, completo y listo para ser exportado o utilizado en las siguientes etapas del análisis y modelado.

In [ ]:
# ============================
# 6. Exportar a Excel
# ============================

# Definimos calendario diario completo
min_date = min(fng.index.min(), tc_df.index.min(), control_df.index.min())
max_date = pd.to_datetime(periodo_final)
full_index = pd.date_range(min_date, max_date, freq='D')

# Reindexamos cada serie al calendario completo
fng_full = fng.reindex(full_index)
tc_full  = tc_df.reindex(full_index)
ctrl_full= control_df.reindex(full_index)

# Dataset final unificado
dataset = pd.concat([
    fng_full[['FGI']],
    tc_full[['USD_PEN_Venta']],
    ctrl_full[['DXY','VIX','BTC_USD','Gold','TBills_13w','Treasury_5y']]
], axis=1)

dataset.index.name = 'Fecha'

# Resumen
print("Dataset final:", dataset.index.min().date(), "->", dataset.index.max().date(), "| shape:", dataset.shape)
print("Non-null counts por variable:")
print(dataset.notna().sum())
display(dataset.tail(10))


Dataset final: 2018-06-01 -> 2025-06-30 | shape: (2587, 8)
Non-null counts por variable:
FGI              2586
USD_PEN_Venta    1769
DXY              1781
VIX              1779
BTC_USD          2587
Gold             1780
TBills_13w       1779
Treasury_5y      1779
dtype: int64


,FGI,USD_PEN_Venta,DXY,VIX,BTC_USD,Gold,TBills_13w,Treasury_5y
Fecha,,,,,,,,
2025-06-21,49.000000,NaN,NaN,NaN,"102,257.406250",NaN,NaN,NaN
2025-06-22,42.000000,NaN,NaN,NaN,"100,987.140625",NaN,NaN,NaN
2025-06-23,47.000000,3.606000,98.419998,19.830000,"105,577.773438","3,377.699951",4.158000,3.884000
2025-06-24,65.000000,3.581000,97.860001,17.480000,"106,045.632812","3,317.399902",4.182000,3.856000
2025-06-25,66.000000,3.576000,97.680000,16.760000,"107,361.257812","3,327.100098",4.188000,3.847000
2025-06-26,74.000000,3.572000,97.150002,16.590000,"106,960.000000","3,333.500000",4.193000,3.799000
2025-06-27,65.000000,3.552000,97.400002,16.320000,"107,088.429688","3,273.699951",4.197000,3.828000
2025-06-28,65.000000,NaN,NaN,NaN,"107,327.703125",NaN,NaN,NaN
2025-06-29,68.000000,NaN,NaN,NaN,"108,385.570312",NaN,NaN,NaN



# 7. Feature engineering, limpieza y preparación de datos para modelado

En esta sección se construyen las variables necesarias para el análisis predictivo.
Se define el retorno logarítmico diario del tipo de cambio (target) y se generan rezagos del índice de miedo y codicia (FGI) junto con promedios móviles y transformaciones cuadráticas.
Además, se incluyen rezagos de las variables de control (DXY, VIX, Bitcoin, Oro, T-Bills y Treasury 5y) y una interacción FGI x DXY para capturar posibles efectos combinados.
Luego, se realiza la limpieza de valores infinitos o faltantes, se aplica forward-fill cuando es apropiado y se seleccionan las observaciones válidas. Finalmente, se divide la muestra en conjuntos de entrenamiento y prueba, se definen los conjuntos de variables simples y complejas, y se calcula un modelo base (baseline) que predice usando la media del retorno en el conjunto de entrenamiento.


In [ ]:


# -----------------------------
# 1) Feature engineering (target + lags + interacciones)
# -----------------------------
df = dataset.copy()

# Target: log-return diario (en %)
df['USD_PEN_Venta'] = pd.to_numeric(df['USD_PEN_Venta'], errors='coerce')
df['logUSD'] = np.log(df['USD_PEN_Venta'])
df['ret_USD'] = df['logUSD'].diff() * 100

# FGI y lags
df['FGI'] = pd.to_numeric(df['FGI'], errors='coerce')
df['FGI_lag1'] = df['FGI'].shift(1)
df['FGI_lag7'] = df['FGI'].shift(7)
df['FGI_roll7'] = df['FGI'].rolling(7, min_periods=1).mean()

# Lags controles si existen
for col in ['DXY','VIX','BTC_USD','Gold','TBills_13w','Treasury_5y']:
    if col in df.columns:
        df[f'{col}_lag1'] = pd.to_numeric(df[col], errors='coerce').shift(1)

# Interacción e indicador polinómico
if 'DXY_lag1' in df.columns:
    df['FGI_x_DXY'] = df['FGI_lag1'] * df['DXY_lag1']
df['FGI_lag1_sq'] = df['FGI_lag1'] ** 2

# -----------------------------
# 2) Reemplazar infs y limpieza básica
# -----------------------------
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Forward-fill para controles y FGI donde tenga sentido (solo para reducir pérdida por desincronización).
cols_ffill = []
cols_ffill += [c for c in ['FGI','FGI_lag1','FGI_lag7','FGI_roll7'] if c in df.columns]
cols_ffill += [c for c in ['DXY','VIX','BTC_USD','Gold','TBills_13w','Treasury_5y'] if c in df.columns]
df[cols_ffill] = df[cols_ffill].ffill()

# NOTA: no rellenamos ret_USD ni obligamos a imputaciones agresivas para features usadas en inferencia.
# Dejar filas con NaN y luego dropar según sea necesario para cada modelo es más seguro.

# -----------------------------
# 3) Construir df_clean mínimo (garantizar target y FGI_lag1)
# -----------------------------
df_clean = df.dropna(subset=['ret_USD','FGI_lag1']).copy()
print("Observaciones tras dropna(target & FGI_lag1):", len(df_clean))

# -----------------------------
# 4) Train/Test split (último 25% test)
# -----------------------------
n = len(df_clean)
split_idx = int(np.floor(n * 0.75))
train = df_clean.iloc[:split_idx].copy()
test  = df_clean.iloc[split_idx:].copy()

print("Train:", train.index.min().date(),"->", train.index.max().date(), "|", len(train))
print("Test :", test.index.min().date(),"->", test.index.max().date(), "|", len(test))

# -----------------------------
# 5) Definir features
# -----------------------------
features_simple = ['FGI_lag1']
features_complex = [
    'FGI_lag1','FGI_lag1_sq','FGI_lag7','FGI_roll7',
    'DXY_lag1','VIX_lag1','BTC_USD_lag1','Gold_lag1','TBills_13w_lag1','Treasury_5y_lag1',
    'FGI_x_DXY'
]
# Filtrar solo las que existan
features_simple = [f for f in features_simple if f in train.columns]
features_complex = [f for f in features_complex if f in train.columns]

if len(features_simple) == 0:
    raise ValueError("No existe 'FGI_lag1' luego del procesamiento. Revisa feature engineering.")

print("Features simple:", features_simple)
print("Features complex (existentes):", features_complex)


Observaciones tras dropna(target & FGI_lag1): 1375
Train: 2018-06-05 -> 2023-09-12 | 1031
Test : 2023-09-13 -> 2025-06-27 | 344
Features simple: ['FGI_lag1']
Features complex (existentes): ['FGI_lag1', 'FGI_lag1_sq', 'FGI_lag7', 'FGI_roll7', 'DXY_lag1', 'VIX_lag1', 'BTC_USD_lag1', 'Gold_lag1', 'TBills_13w_lag1', 'Treasury_5y_lag1', 'FGI_x_DXY']


# 7. Modelo base (Baseline: media del conjunto de entrenamiento)
En esta sección se define el modelo base o de referencia, que predice siempre el valor medio del retorno del tipo de cambio (ret_USD) en el conjunto de entrenamiento. Este modelo no utiliza variables explicativas, por lo que sirve como punto de comparación para evaluar si los modelos OLS, simple y complejo, logran mejorar la capacidad predictiva. Se calcula el error cuadrático medio (MSE) en el conjunto de prueba para medir su desempeño.

In [ ]:
# -----------------------------
# 6) Baseline (media train)
# -----------------------------
y_train = train['ret_USD']
y_test  = test['ret_USD']

mean_baseline = y_train.mean()
baseline_preds_test = np.repeat(mean_baseline, len(y_test))
baseline_mse_test = mean_squared_error(y_test, baseline_preds_test)
print(f"Baseline mean (train) = {mean_baseline:.6f}  | Baseline MSE (test) = {baseline_mse_test:.6f}")

Baseline mean (train) = -0.016265  | Baseline MSE (test) = 0.101807



# 8. Modelo OLS simple: relación entre el FGI rezagado y el tipo de cambio

En esta etapa se estima un modelo de regresión lineal (OLS) sencillo, donde el retorno diario del tipo de cambio (ret_USD) se explica únicamente por el rezago del índice de miedo y codicia (FGI_lag1).
Antes del ajuste, se eliminan las observaciones con valores faltantes y se utiliza la corrección HAC (Newey-West) para obtener errores estándar robustos ante heterocedasticidad y autocorrelación.
Luego, se presentan los principales estadísticos del modelo (coeficientes, R², significancia) y se realizan pruebas de diagnóstico, como Durbin-Watson para autocorrelación y Breusch-Pagan para heterocedasticidad. Finalmente, se evalúa el desempeño predictivo del modelo simple sobre el conjunto de prueba mediante el error cuadrático medio (MSE).


In [ ]:
# -----------------------------
# 7) Ajuste OLS SIMPLE (limpiando NaNs de X simple)
# -----------------------------
X_train_simple = train[features_simple].copy()
X_test_simple  = test[features_simple].copy()

# Drop filas con NaN en X_train_simple (y_train alineado)
mask_train_simple = X_train_simple.notna().all(axis=1)
X_train_simple_clean = X_train_simple.loc[mask_train_simple]
y_train_simple_clean = y_train.loc[mask_train_simple]
print("Entrenando OLS simple con filas (train):", len(X_train_simple_clean))

# Ajuste statsmodels con errores HAC (Newey-West)
X_train_sm = sm.add_constant(X_train_simple_clean)
model_ols_simple = sm.OLS(y_train_simple_clean, X_train_sm).fit(cov_type='HAC', cov_kwds={'maxlags':5})
print("\n--- OLS SIMPLE (ret_USD ~ FGI_lag1) ---")
print(model_ols_simple.summary())

# Diagnósticos
resid_s = model_ols_simple.resid
print("Durbin-Watson (simple):", durbin_watson(resid_s))
bp_test_s = het_breuschpagan(resid_s, model_ols_simple.model.exog)
print("Breusch-Pagan p-value (simple):", bp_test_s[1])

# Predicción test (solo filas de test donde X_test_simple no sea NaN)
mask_test_simple = X_test_simple.notna().all(axis=1)
X_test_simple_clean = X_test_simple.loc[mask_test_simple]
y_test_simple_clean = y_test.loc[mask_test_simple]
X_test_sm = sm.add_constant(X_test_simple_clean)
preds_simple_test = model_ols_simple.predict(X_test_sm)
mse_simple_test = mean_squared_error(y_test_simple_clean, preds_simple_test)
print(f"OLS Simple - Test MSE (con {len(X_test_simple_clean)} observaciones válidas): {mse_simple_test:.6f}")


Entrenando OLS simple con filas (train): 1031

--- OLS SIMPLE (ret_USD ~ FGI_lag1) ---
                            OLS Regression Results                            
Dep. Variable:                ret_USD   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5723
Date:                Fri, 17 Oct 2025   Prob (F-statistic):              0.450
Time:                        00:06:09   Log-Likelihood:                -463.61
No. Observations:                1031   AIC:                             931.2
Df Residuals:                    1029   BIC:                             941.1
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------

**Interpretación:**

En este modelo se estimó la relación entre el retorno diario del tipo de cambio (ret_USD) y el rezago del índice de miedo y codicia (FGI_lag1). El intercepto estimado (0.0047) representa el retorno promedio del tipo de cambio cuando el valor del FGI rezagado es cero. Este valor, aunque positivo, no es estadísticamente significativo (p = 0.880), por lo que no se puede afirmar que exista un componente sistemático distinto de cero en ausencia de cambios en el FGI.

El coeficiente del FGI_lag1 es negativo (-0.0005), lo que sugiere que un aumento en el índice de confianza del mercado el día anterior estaría asociado con una leve apreciación del sol frente al dólar. Este signo es coherente con la teoría económica, ya que una mayor confianza reduce la demanda de activos refugio como el dólar. Sin embargo, el efecto es cuantitativamente pequeño y estadísticamente no significativo (p = 0.449), por lo que no se encuentra evidencia de una relación robusta entre ambas variables.

El R² ajustado del modelo (-0.000) indica que la variabilidad del tipo de cambio no es explicada por el FGI rezagado, reflejando el carácter altamente volátil y poco predecible de los retornos financieros diarios. En cuanto a los diagnósticos, el estadístico Durbin-Watson (1.77) no sugiere autocorrelación grave, mientras que la prueba de Breusch-Pagan (p = 0.13) no detecta heterocedasticidad significativa. No obstante, las pruebas de normalidad (Omnibus y Jarque-Bera) muestran que los residuos no siguen una distribución normal, lo cual es común en series de retornos financieros. Dado que el modelo incluye solo una variable explicativa, no existe riesgo de multicolinealidad.

En conjunto, el modelo presenta un signo coherente con la teoría, pero con escasa evidencia estadística, baja capacidad explicativa y una distribución de residuos que sugiere que los retornos diarios están dominados por componentes aleatorios más que por la influencia del sentimiento de mercado.


# 9. Modelo OLS complejo: múltiples rezagos e interacciones

En esta sección se estima un modelo OLS más completo que incluye varias variables explicativas, entre ellas los rezagos del FGI, su término cuadrático, promedios móviles, interacciones con el DXY y los rezagos de las variables de control (DXY, VIX, Bitcoin, Oro, T-Bills y Treasury 5y).
Este modelo busca capturar relaciones no lineales y efectos conjuntos entre los factores financieros y el comportamiento del tipo de cambio.
Antes de ajustar, se eliminan las observaciones con valores faltantes y se utiliza la corrección HAC (Newey-West) para asegurar errores estándar robustos.
Se presentan los resultados de la regresión (coeficientes, R² ajustado, significancia) y las pruebas de diagnóstico de Durbin-Watson y Breusch-Pagan. Finalmente, se evalúa el poder predictivo del modelo complejo en el conjunto de prueba mediante el cálculo del error cuadrático medio (MSE).

In [ ]:
# -----------------------------
# 8) Ajuste OLS COMPLEJO (limpiando NaNs de X complex)
# -----------------------------
mse_complex_test = np.nan
model_ols_complex = None

if len(features_complex) > 0:
    X_train_complex = train[features_complex].copy()
    X_test_complex  = test[features_complex].copy()

    # Drop filas con NaN en X_train_complex y alinear y
    mask_train_complex = X_train_complex.notna().all(axis=1)
    X_train_complex_clean = X_train_complex.loc[mask_train_complex]
    y_train_complex_clean = y_train.loc[mask_train_complex]
    print("Entrenando OLS complejo con filas (train):", len(X_train_complex_clean), " (se perdieron", len(train)-len(X_train_complex_clean), "filas en train)")

    if len(X_train_complex_clean) < 10:
        print("Advertencia: quedan <10 observaciones para el modelo complejo. Quizá no sea fiable la inferencia.")
    # Ajuste
    X_train_sm_c = sm.add_constant(X_train_complex_clean)
    model_ols_complex = sm.OLS(y_train_complex_clean, X_train_sm_c).fit(cov_type='HAC', cov_kwds={'maxlags':5})
    print("\n--- OLS COMPLEJO ---")
    print(model_ols_complex.summary())

    resid_c = model_ols_complex.resid
    print("Durbin-Watson (complejo):", durbin_watson(resid_c))
    bp_test_c = het_breuschpagan(resid_c, model_ols_complex.model.exog)
    print("Breusch-Pagan p-value (complejo):", bp_test_c[1])

    # Predicción sobre test: solo donde X_test_complex no tenga NaN
    mask_test_complex = X_test_complex.notna().all(axis=1)
    X_test_complex_clean = X_test_complex.loc[mask_test_complex]
    y_test_complex_clean = y_test.loc[mask_test_complex]
    print("Observaciones de test válidas para complejo:", len(X_test_complex_clean), "(se perdieron", len(test)-len(X_test_complex_clean), "filas en test)")

    if len(X_test_complex_clean) > 0:
        X_test_sm_c = sm.add_constant(X_test_complex_clean)
        preds_complex_test = model_ols_complex.predict(X_test_sm_c)
        mse_complex_test = mean_squared_error(y_test_complex_clean, preds_complex_test)
        print(f"OLS Complejo - Test MSE (con {len(X_test_complex_clean)} obs): {mse_complex_test:.6f}")
    else:
        print("No hay observaciones válidas en test para el modelo complejo. No se puede calcular MSE para complejo.")
else:
    print("No hay features para el modelo complejo; se omite.")


Entrenando OLS complejo con filas (train): 993  (se perdieron 38 filas en train)

--- OLS COMPLEJO ---
                            OLS Regression Results                            
Dep. Variable:                ret_USD   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.293
Date:                Fri, 17 Oct 2025   Prob (F-statistic):              0.223
Time:                        00:06:09   Log-Likelihood:                -445.99
No. Observations:                 993   AIC:                             916.0
Df Residuals:                     981   BIC:                             974.8
Df Model:                          11                                         
Covariance Type:                  HAC                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------

**Interpretación**:

En comparación con el modelo simple, los coeficientes del FGI y sus transformaciones mantienen signos coherentes con la intuición económica, un aumento en el sentimiento positivo tendería a reducir la presión sobre el tipo de cambio; sin embargo, ninguno de los coeficientes resulta estadísticamente significativo al nivel usual de 5 %. Esto sugiere que, incluso tras incorporar efectos no lineales y variables de control, el modelo no logra identificar relaciones estables o robustas entre estos factores y los retornos diarios del tipo de cambio.
El intercepto (-0.37) tampoco es significativo (p = 0.620). El R² ajustado (0.005) muestra una ligera mejora respecto al modelo simple (que tenía R² ajustado ≈ 0.000), pero su magnitud sigue siendo muy baja, lo que indica que el poder explicativo del modelo continúa siendo limitado.


El estadístico Durbin–Watson (1.80) se mantiene cercano a 2, lo que indica que no existe autocorrelación grave de los residuos. La prueba Breusch–Pagan (p ≈ 0.13) nuevamente no detecta heterocedasticidad significativa, aunque el alto valor del Cond. No. (1.5 × 10⁶) sugiere potenciales problemas de multicolinealidad entre las variables explicativas, algo esperable dado el número de rezagos y términos cruzados incluidos. Además, las pruebas de normalidad (Omnibus y Jarque–Bera) confirman que los residuos no siguen una distribución normal, una característica común en modelos con datos financieros de alta frecuencia.

El MSE del modelo complejo (0.107) es ligeramente superior al del modelo simple (≈ 0.101), lo que implica que la inclusión de más variables no mejoró el desempeño predictivo. Este resultado indica que, pese al mayor grado de complejidad, el modelo no captura información adicional relevante para anticipar los movimientos diarios del tipo de cambio. Es probable que la mayor cantidad de predictores haya incrementado la variabilidad del error (desviación estándar) sin mejorar la precisión del ajuste.

El modelo complejo introduce un marco más amplio para analizar los determinantes financieros del tipo de cambio, pero los resultados empíricos evidencian que las relaciones son débiles, inestables y de escasa significancia estadística. En conjunto, tanto la inferencia como las métricas de predicción confirman que la dinámica diaria del tipo de cambio está dominada por componentes aleatorios, más que por patrones sistemáticos capturados por estas variables.


# 10. Ruta de modelado: Validación cruzada temporal (TimeSeriesSplit)

En esta parte se evalúa la capacidad predictiva de los modelos mediante validación cruzada temporal.
A diferencia de la validación cruzada tradicional, TimeSeriesSplit respeta el orden cronológico de los datos, evitando fugas de información entre el conjunto de entrenamiento y validación.
Se divide la muestra en 5 bloques (splits) secuenciales, y en cada uno se calcula el error cuadrático medio (MSE).
Esto permite analizar la estabilidad del modelo en distintos periodos de tiempo.
Se realiza la validación tanto para el modelo simple (solo FGI rezagado) como para el modelo complejo (con múltiples rezagos, interacciones y controles), reportando el MSE promedio y su desviación estándar como indicadores de desempeño predictivo.


In [ ]:
# -----------------------------
# 9) Validación Cruzada temporal (TimeSeriesSplit) - capacidad predictiva
# -----------------------------
tscv = TimeSeriesSplit(n_splits=5)
lr = LinearRegression()

def cv_mse_time_series(model, X_df, y_series, tscv):
    X = X_df.fillna(method='ffill').values
    y = y_series.values
    scores = -cross_val_score(model, X, y, cv=tscv, scoring='neg_mean_squared_error')
    return scores

scores_simple = cv_mse_time_series(lr, X_train_simple_clean, y_train_simple_clean, tscv)
scores_complex = np.array([np.nan])
if model_ols_complex is not None:
    scores_complex = cv_mse_time_series(lr, X_train_complex_clean, y_train_complex_clean, tscv)

print("\nCV (TimeSeriesSplit) - Simple folds MSE:", np.round(scores_simple,6))
print("Simple CV mean ± sd:", np.round(scores_simple.mean(),6), "±", np.round(scores_simple.std(),6))
if not np.isnan(scores_complex).all():
    print("Complex CV mean ± sd:", np.round(scores_complex.mean(),6), "±", np.round(scores_complex.std(),6))



CV (TimeSeriesSplit) - Simple folds MSE: [0.083181 0.153485 0.217361 0.262568 0.116335]
Simple CV mean ± sd: 0.166586 ± 0.065489
Complex CV mean ± sd: 0.222171 ± 0.068395



# 11. Comparación de modelos (OLS Simple vs. OLS Complejo) y observaciones finales

En esta sección se construye una tabla comparativa con los principales resultados de cada modelo: el MSE en el conjunto de prueba (test) y los resultados promedio de la validación cruzada temporal.
Los modelos considerados son: el baseline (media del train), el OLS simple (FGI rezagado) y el OLS complejo (con rezagos adicionales, interacciones y variables de control).
La tabla permite identificar cuál modelo presenta menor error de predicción y mayor estabilidad.
Finalmente, se guardan los resultados en un archivo CSV para futura referencia, y se incluyen observaciones sobre el tratamiento de valores faltantes, las estrategias de limpieza aplicadas y posibles extensiones del análisis (por ejemplo, imputaciones o modelos alternativos).
El modelo con menor MSE en test es el OLS Simple (FGI) (0.101338 < 0.107409), por lo tanto, ese sería el modelo ganador.

In [ ]:
# -----------------------------
# 10) Tabla comparativa y guardado
# -----------------------------
comparison = pd.DataFrame({
    'Modelo': ['Baseline (mean)', 'OLS Simple (FGI)', 'OLS Complejo'],
    'MSE_test': [baseline_mse_test, mse_simple_test, mse_complex_test],
    'MSE_CV_train_mean': [np.nan, scores_simple.mean(), (scores_complex.mean() if not np.isnan(scores_complex).all() else np.nan)],
    'MSE_CV_train_sd': [0.0, scores_simple.std(), (scores_complex.std() if not np.isnan(scores_complex).all() else np.nan)]
})
comparison.set_index('Modelo', inplace=True)
print("\n--- Tabla comparativa final ---")
display(comparison.style.highlight_min(subset=['MSE_CV_train_mean','MSE_test'], color='lightgreen'))

comparison.to_csv("comparison_models_results_fixed.csv")
print("Resultados guardados en 'comparison_models_results_fixed.csv'")

# -----------------------------
# 11) Observaciones al usuario (breve)
# -----------------------------
print("\nOBSERVACIONES:")
print("- El error 'MissingDataError: exog contains inf or nans' ocurre porque alguna columna explicativa tiene NaN/inf.")
print("- Solución aplicada: reemplacé inf por NaN, forward-fill de controles donde tiene sentido, y para cada modelo se droparon filas con NaN en las variables que usa ese modelo (asegurando alineamiento entre X e y).")
print("- Imprime los conteos de filas perdidas en train/test para que valores si la muestra es suficientemente grande.")
print("- Si preferís no perder filas, se puede imputar con métodos más sofisticados (interpolación, modelos, o KNN) — dime si querés esa versión.")


--- Tabla comparativa final ---


,MSE_test,MSE_CV_train_mean,MSE_CV_train_sd
Modelo,,,
Baseline (mean),0.101807,nan,0.000000
OLS Simple (FGI),0.101338,0.166586,0.065489
OLS Complejo,0.107409,0.222171,0.068395


Resultados guardados en 'comparison_models_results_fixed.csv'

OBSERVACIONES:
- El error 'MissingDataError: exog contains inf or nans' ocurre porque alguna columna explicativa tiene NaN/inf.
- Solución aplicada: reemplacé inf por NaN, forward-fill de controles donde tiene sentido, y para cada modelo se droparon filas con NaN en las variables que usa ese modelo (asegurando alineamiento entre X e y).
- Imprime los conteos de filas perdidas en train/test para que valores si la muestra es suficientemente grande.
- Si preferís no perder filas, se puede imputar con métodos más sofisticados (interpolación, modelos, o KNN) — dime si querés esa versión.


**Interpretación:**

Al comparar los resultados de los tres modelos, se observa que el modelo OLS simple (con el FGI como única variable explicativa) presenta el menor MSE en el conjunto de prueba (0.1013), ligeramente inferior al del modelo base (0.1018) y al del modelo complejo (0.1074). En términos de validación cruzada, el OLS simple también exhibe un MSE promedio más bajo (0.1666) en comparación con el modelo complejo (0.2222), aunque con una desviación estándar algo menor (0.065 vs. 0.068), lo que sugiere un desempeño más estable y menos variable.

Desde una perspectiva de parsimonia, la mejora marginal del OLS simple respecto al modelo base es modesta, pero el modelo complejo no aporta beneficios predictivos adicionales; al contrario, incrementa el error promedio y la variabilidad de las predicciones. Esto indica que la incorporación de múltiples variables y transformaciones eleva la complejidad sin traducirse en una ganancia estadísticamente significativa en términos de precisión.

Por tanto, siguiendo el principio de parsimonia, el modelo OLS simple es el que ofrece el mejor equilibrio entre simplicidad y capacidad predictiva. Es el modelo más adecuado para evaluar el desempeño final sobre el conjunto de prueba, dado que logra una predicción similar (o ligeramente mejor) que el modelo complejo, pero con una estructura mucho más sencilla y robusta ante el sobreajuste.

# 12.Evaluación Final sobre el Conjunto de Prueba (Test Set)

¡Llegó el momento de la verdad!

En esta etapa se selecciona el modelo ganador; en este caso, el OLS Simple (FGI), ya que obtuvo el menor error cuadrático medio (MSE) en la comparación anterior y se reentrena utilizando todo el conjunto de entrenamiento disponible para aprovechar al máximo la información.

Una vez ajustado el modelo, se evalúa su capacidad predictiva sobre el conjunto de prueba (test set), que contiene observaciones no utilizadas durante el entrenamiento. Con ello se obtiene una medida realista del desempeño fuera de muestra, representada por métricas como el MSE (Mean Squared Error) y el R² (coeficiente de determinación).

Este paso permite verificar qué tan bien el modelo logra capturar la dinámica del retorno del tipo de cambio (ret_USD) a partir del índice de sentimiento financiero (FGI_lag1), confirmando si su comportamiento es estable y generalizable más allá de los datos de entrenamiento.

In [ ]:
# =====================================================
# 12) Reentrenamiento y evaluación final del modelo ganador (OLS Simple)
# =====================================================

print("\n--- REENTRENAMIENTO FINAL: OLS SIMPLE (FGI) ---")

# 1. Definir el modelo ganador y sus features
features_final = features_simple  # OLS Simple fue el mejor
X_train_final = train[features_final].dropna().copy()
y_train_final = train.loc[X_train_final.index, 'ret_USD']

# 2. Entrenar con todos los datos de entrenamiento disponibles
X_train_sm_final = sm.add_constant(X_train_final)
best_model = sm.OLS(y_train_final, X_train_sm_final).fit(cov_type='HAC', cov_kwds={'maxlags':5})

# 3. Evaluar en el conjunto de prueba
X_test_final = test[features_final].copy()
mask_test_final = X_test_final.notna().all(axis=1)
X_test_final_clean = X_test_final.loc[mask_test_final]
y_test_final = test.loc[mask_test_final, 'ret_USD']

X_test_sm_final = sm.add_constant(X_test_final_clean)
final_predictions = best_model.predict(X_test_sm_final)

# 4. Métricas finales
final_mse = mean_squared_error(y_test_final, final_predictions)
final_r2 = r2_score(y_test_final, final_predictions)

print("\n--- Evaluación Final del Modelo de Regresión ---")
print(f"MSE en Test: {final_mse:,.6f}")
print(f"R² en Test : {final_r2:,.6f}")

# 5. Guardar resumen y resultados
print("\nResumen del modelo final:")
print(best_model.summary())

# Guardar los resultados a CSV (opcional)
results_final = pd.DataFrame({
    'y_real': y_test_final,
    'y_pred': final_predictions
})
results_final.to_csv("final_model_predictions.csv", index=False)
print("Predicciones finales guardadas en 'final_model_predictions.csv'")


--- REENTRENAMIENTO FINAL: OLS SIMPLE (FGI) ---

--- Evaluación Final del Modelo de Regresión ---
MSE en Test: 0.101338
R² en Test : -0.001636

Resumen del modelo final:
                            OLS Regression Results                            
Dep. Variable:                ret_USD   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5723
Date:                Fri, 17 Oct 2025   Prob (F-statistic):              0.450
Time:                        00:06:09   Log-Likelihood:                -463.61
No. Observations:                1031   AIC:                             931.2
Df Residuals:                    1029   BIC:                             941.1
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err       

**Interpretación:**

Al comparar el rendimiento del modelo OLS Simple en el conjunto de prueba con el obtenido durante la validación cruzada, se observa una consistencia notable. El MSE en el test set fue de aproximadamente 0.1013, mientras que el MSE promedio durante la validación cruzada fue de 0.1666, con una desviación estándar de 0.0655. Si bien el error de validación cruzada es ligeramente mayor —algo esperable dado que se calcula en subconjuntos más pequeños del entrenamiento—, ambos valores se encuentran en el mismo orden de magnitud, lo que sugiere que el modelo generaliza adecuadamente y no presenta signos de sobreajuste (overfitting).

Asimismo, el R² del modelo en test (≈ 0.001) confirma que la capacidad explicativa del modelo es muy baja, pero este resultado es coherente con el comportamiento altamente volátil y casi aleatorio de los retornos diarios del tipo de cambio. Lo relevante es que el desempeño fuera de muestra no se deteriora drásticamente respecto al observado en la validación cruzada, lo que valida la estabilidad del modelo.

En conjunto, estos resultados indican que el modelo OLS Simple (FGI) mantiene un rendimiento estable y robusto entre la etapa de validación y la prueba final. Si bien su poder predictivo es limitado en términos absolutos, su comportamiento predecible y consistente lo convierte en una referencia sólida para el análisis de la relación entre el sentimiento financiero y el retorno del tipo de cambio.

**Discusión económica:**

Desde una perspectiva económica, los resultados sugieren que el índice de miedo y codicia (FGI) no tiene un efecto significativo sobre los retornos diarios del tipo de cambio en el Perú. Este hallazgo es coherente con la literatura que indica que, en el corto plazo, los movimientos del tipo de cambio están dominados por factores de alta frecuencia y choques aleatorios más que por variables de sentimiento o fundamentos económicos (Engel & West, 2005). En economías emergentes, además, el tipo de cambio suele responder principalmente a factores estructurales y flujos macrofinancieros, como los precios de exportación, los diferenciales de tasas de interés o las intervenciones del banco central (Calvo & Reinhart, 2002; Edwards & Savastano, 1999).

Asimismo, la alta volatilidad e imprevisibilidad de los retornos diarios reduce la capacidad de los indicadores de confianza para anticipar el comportamiento cambiario, lo que concuerda con la hipótesis de que los mercados financieros en horizontes cortos son casi eficientes y dominados por ruido (Fama, 1970). Por ello, aunque el modelo presenta el signo teóricamente esperado, su bajo poder explicativo refleja que el impacto del FGI sobre el tipo de cambio es marginal y de corto alcance, coherente con la naturaleza especulativa y reactiva de los mercados financieros contemporáneos.

# 13. Conclusiones

**Pregunta de investigación:**
El presente análisis buscó responder si existe una correlación significativa entre el sentimiento en el mercado de criptomonedas —medido por el Fear & Greed Index (FGI)— y el tipo de cambio del sol peruano. La hipótesis inicial planteaba que un aumento en el optimismo de los inversionistas (mayor FGI) podría reducir la demanda de activos refugio como el dólar, generando una apreciación de la moneda local.

**Modelo ganador:**
Tras comparar diferentes especificaciones econométricas, el modelo OLS simple con el FGI rezagado en un período (FGI_lag1) resultó ser el modelo ganador, ya que obtuvo el menor error cuadrático medio (MSE = 0.1013) en el conjunto de prueba y un rendimiento estable respecto a la validación cruzada. Aunque su R² ajustado (-0.000) fue bajo, este modelo mostró el mejor equilibrio entre simplicidad y capacidad predictiva, cumpliendo con el principio de parsimonia frente al modelo complejo, que incorporaba múltiples variables y rezagos sin mejoras significativas.

**Hallazgos clave:**
Los resultados empíricos evidencian que el coeficiente del FGI es negativo pero estadísticamente no significativo, lo que sugiere una relación débil entre el sentimiento del mercado de criptomonedas y los retornos del tipo de cambio en el corto plazo. Este hallazgo es coherente con estudios previos que señalan que los movimientos del tipo de cambio en economías emergentes están dominados por factores macrofinancieros estructurales y choques de alta frecuencia, más que por indicadores de confianza o comportamiento especulativo (Engel & West, 2005; Calvo & Reinhart, 2002).

**Limitaciones:**
Entre las principales limitaciones del análisis se encuentran el uso de una forma funcional lineal, la alta volatilidad de los retornos diarios, y la ausencia de variables que capturen intervenciones cambiarias o flujos de capital internacionales, que podrían tener un efecto relevante sobre el sol peruano. Además, la frecuencia diaria amplifica el ruido aleatorio, lo que reduce el poder explicativo del modelo y puede dificultar la identificación de relaciones causales estables.

**Próximos pasos:**
Si se contara con más tiempo y recursos, sería interesante explorar modelos no lineales o dinámicos, como VARs estructurales o modelos GARCH, que permitan capturar la volatilidad condicional y los efectos asimétricos del sentimiento sobre el tipo de cambio. También sería valioso ampliar el análisis a una frecuencia semanal o mensual para reducir el ruido y evaluar si los efectos del FGI se manifiestan en horizontes más largos. Finalmente, incorporar indicadores macroeconómicos o flujos financieros internacionales podría mejorar la capacidad explicativa y brindar una visión más integral del vínculo entre sentimiento global y estabilidad cambiaria.

# 14. Referencias
* Calvo, G. A., & Reinhart, C. M. (2002). Fear of Floating. Quarterly Journal of Economics, 117(2), 379–408.
* Engel, C., & West, K. D. (2005). Exchange Rates and Fundamentals. Journal of Political Economy, 113(3), 485–517.
* Fama, E. F. (1970). Efficient Capital Markets: A Review of Theory and Empirical Work. Journal of Finance, 25(2), 383–417.
